In [40]:
from ai71 import AI71
from dotenv import load_dotenv
import os
import json
import re
from extract_pdf import extract_text_from_pdf, extract_text_from_pdfs, clean_text

In [41]:
load_dotenv()
api_key = os.getenv('API_KEY')

In [42]:
client = AI71(api_key)

## Pdf Extraction

In [43]:
text = extract_text_from_pdf("/home/space/Downloads/test.pdf")

or

In [44]:
multiple_texts = extract_text_from_pdfs(["/home/space/Downloads/test.pdf", "/home/space/Downloads/test.pdf"])

In [45]:
if len(text) > 4800:
    print("Text is too long")

Text is too long


In [46]:

text = text.replace("\n", " ")
text = re.sub(r'\s{2,}', ' ', text)

if len(text) > 4500:
    text_list = [text[i:i+4000] for i in range(0, len(text), 4000)]
    

In [47]:
print(len(text_list[1]))
print(clean_text(text_list[1]))
test = [text_list[0], text_list[0], text_list[0]]


4000
extreme 19.3% → .193 •since greater significance level (0.05) fail reject null hypothesis (reject coin fair). •there enough evidence reject coin fair. 3 1744 50 56 𝑝=0.193 •: null hypothesis (h0), probability getting sample extreme expectation (alternative hypothesis, h1). •example: null hypothesis coin fair (h0), alternative hypothesis (h1) coin fair •if coin fair, probability getting 60 heads sample extreme 3.2% → .032 •since smaller significance level (0.05) reject null hypothesis: •< alpha → reject null hypothesis. 3 1850 60 ref 𝑝=0. •if coin fair, probability getting heads sample extreme 0.0% → .00 •something wrong coin. reject null hypothesis, 3 1950 ref𝑝=0.000motivation gradient calculation •in machine learning implementation, often objective function want minimize iteratively. •we calculate gradient find minimum. 3 20what gradient? •derivative function 𝑓 direction fastest increase 𝑓 •one unit step 𝑥 direction increases 𝑓(𝑥) approximately 𝜕𝑓 𝜕𝑥 units 𝑥 direction. •gradient 

## Lesson topic generation

In [48]:
def create_lesson_topics(text_list):
    lesson_topics = []
    if len(text_list) == 1:
        response = client.chat.completions.create(
        model="tiiuae/falcon-180B-chat",
        messages=[
            {"role": "system", "content": "You are a lesson planner."},
            {"role": "user", "content": text_list},
            {"role": "user", "content": "Create 3 small lesson topics to teach the given material in json format giving a proper explanation on what the lesson should teach."},
        ],
        )
        lesson_topics.append(response)
    else:
        response = client.chat.completions.create(
            model="tiiuae/falcon-180B-chat",
            messages=[
                {"role": "system", "content": "You are a lesson planner."},
                {"role": "user", "content": text_list[0]},
                {"role": "user", "content": "Suggest 2 small lesson topics that can be taught from the given material and give a proper explanation on what the lesson should teach. Please provide the topics in json format like this: [{'topic': 'topic name', 'explanation': 'explanation of the topic'}]"},
            ],
            )
        lesson_topics.append(response)
        for i in text_list[1:len(text_list)]:
            
            response = client.chat.completions.create(
            model="tiiuae/falcon-180B-chat",
            messages=[
                {"role": "system", "content": "You are a lesson planner."},
                {"role": "user", "content": i},
                {"role": "user", "content": f"{response} are the topics which have already been covered (please do not suggest them or their related subtopics). Suggest 2 more topics to teach the given material in json format giving a proper explanation on what the lesson should teach. The json format should be like this: [{{'topic': 'topic name', 'explanation': 'explanation of the topic'/}}]. Please only respond with the new topics and their explanations in json, not the old ones."},
            ],
            )
            lesson_topics.append(response)
    return lesson_topics

I WILL BLOW UPPPPP.
text_list[0] works but text_[1] doesnt work. Prolly an issue with the text formatting. 
Fix that and then you can give it chunks and ask it to create the topics in chunks

In [49]:
response = create_lesson_topics(test)

In [50]:
# Function to clean and fix the JSON strings
def fix_json_string(json_string):
    # Removing leading and trailing spaces and newlines
    json_string = json_string.strip()
    # Adding brackets if they are missing
    if not json_string.startswith('['):
        json_string = '[' + json_string
    if not json_string.endswith(']'):
        json_string = json_string + ']'
    # Ensure proper comma separation between objects
    json_string = json_string.replace('}\n', '},\n').replace('},\n]', '}\n]')
    return json_string

In [51]:
json_strings = [resp.choices[0].message.content.replace("User:", "") for resp in response]

for i in json_strings:
    print(i)

 [
  {
    "topic": "Calculus Overview",
    "explanation": "This lesson will provide an overview of calculus, including vectorization, linear algebra, and statistics. It will cover the basics of vectorization, linear algebra, and statistics, and how they are used in machine learning. The lesson will also introduce the concept of calculus and its importance in machine learning."
  },
  {
    "topic": "Hypothesis Testing",
    "explanation": "This lesson will focus on hypothesis testing, including the definition, process, and meaning of significant level and p-value. It will also provide an example of how to conduct a hypothesis test and interpret the results. The lesson will help students understand how to make inferences about a population based on sample data."
  }
]
 [
  {
    "topic": "Vectorization",
    "explanation": "This lesson will cover the process of converting data into a vector format, which is necessary for machine learning algorithms to understand and process the data. 

In [52]:

all_lessons = []

# Iterate through each JSON string, fix the formatting, parse it, and extend the all_lessons list
for json_str in json_strings:
    try:
        fixed_json_str = fix_json_string(json_str)
        lessons = json.loads(fixed_json_str)
        all_lessons.extend(lessons)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")

# Merge the dictionaries into a single dictionary
merged_dict = {lesson['topic']: lesson['explanation'] for lesson in all_lessons}



In [53]:
Giga = {
    "lesson_topics": merged_dict,
    "lessons": {},
}


for key, value in Giga["lesson_topics"].items():
    print(f"{key}: {value}")

Calculus Overview: This lesson will provide an overview of calculus, including vectorization, linear algebra, and statistics. It will cover the basics of vectorization, linear algebra, and statistics, and how they are used in machine learning. The lesson will also introduce the concept of calculus and its importance in machine learning.
Hypothesis Testing: This lesson will focus on hypothesis testing, including the definition, process, and meaning of significant level and p-value. It will also provide an example of how to conduct a hypothesis test and interpret the results. The lesson will help students understand how to make inferences about a population based on sample data.
Vectorization: This lesson will cover the process of converting data into a vector format, which is necessary for machine learning algorithms to understand and process the data. It will also cover the different types of vectorization, such as one-hot encoding and binary encoding, and how they are used in differen

## Lesson Generators

In [54]:
def create_lesson(topic, explanation):
    response = client.chat.completions.create(
        model="tiiuae/falcon-180B-chat",
        messages=[
            {"role": "system", "content": "You are a great teacher bot. You create emersive and long lessons for students. They are straight to the point and not addressed to anyone. You are writing long and extensive course material that is easy to understand."},
            {"role": "user", "content": f"Write a long, detailed and extensive overview for '{topic}'. It should be structured like a teacher explaining the topic to a class, covering all relevant aspects including fundamental principles, key concepts, detailed explanations, practical applications, and in-depth examples. {explanation}"},
        ],
    )
    return response

In [55]:
def create_lessons(lesson_topics):
    lessons = {}
    for topic, explanation in lesson_topics.items():
        print(f"Creating lesson for topic: {topic}")
        response = create_lesson(topic, explanation)
        print(f"Lesson created for topic: {topic}")
        lessons[topic] = response.choices[0].message.content
    return lessons

In [56]:
BRUH = create_lessons(Giga["lesson_topics"])

Creating lesson for topic: Calculus Overview
Lesson created for topic: Calculus Overview
Creating lesson for topic: Hypothesis Testing
Lesson created for topic: Hypothesis Testing
Creating lesson for topic: Vectorization
Lesson created for topic: Vectorization
Creating lesson for topic: Linear Algebra
Lesson created for topic: Linear Algebra
Creating lesson for topic: Calculus
Lesson created for topic: Calculus
Creating lesson for topic: Statistics
Lesson created for topic: Statistics


In [59]:
print(BRUH.keys())

dict_keys(['Calculus Overview', 'Hypothesis Testing', 'Vectorization', 'Linear Algebra', 'Calculus', 'Statistics'])


In [105]:
Giga["lessons"] = BRUH

In [108]:
print(Giga["lesson_topics"].keys())

dict_keys(['Calculus Overview', 'Hypothesis Testing', 'Vectorization', 'Linear Algebra', 'Calculus', 'Statistics'])


In [61]:
print(BRUH['Hypothesis Testing'])

 Hypothesis Testing: A Comprehensive Overview

Introduction:

Hypothesis testing is a statistical method used to make inferences about a population based on sample data. It is a crucial tool in research and decision-making processes, as it allows us to draw conclusions about a population based on a sample. In this lesson, we will explore the fundamental principles of hypothesis testing, key concepts, detailed explanations, practical applications, and in-depth examples.

Fundamental Principles:

Hypothesis testing is based on the idea that we can make assumptions about a population based on a sample. The process involves testing a hypothesis, which is a statement about a population parameter. The hypothesis is tested using sample data, and the results are used to make inferences about the population.

There are two types of hypotheses: the null hypothesis and the alternative hypothesis. The null hypothesis is a statement that assumes there is no difference between the sample and the pop

In [68]:
topics = list(Giga["lesson_topics"].keys())
topic = topics[1]
explanation = Giga["lesson_topics"][topic]

response = create_lesson(topic, explanation)

In [69]:
print(response.usage)

completion_tokens=1189 prompt_tokens=168 total_tokens=1357


In [70]:
lesson = response.choices[0].message.content
print(lesson)

 Hypothesis Testing: A Comprehensive Overview

Introduction:

Hypothesis testing is a statistical method used to make inferences about a population based on sample data. It is a crucial tool in research and decision-making processes, as it allows us to draw conclusions about a population based on a sample. In this lesson, we will explore the fundamental principles of hypothesis testing, key concepts, detailed explanations, practical applications, and in-depth examples.

Fundamental Principles:

Hypothesis testing is based on the idea that we can make assumptions about a population based on a sample. The process involves testing a hypothesis, which is a statement about a population parameter. The hypothesis is tested using sample data, and the results are used to make inferences about the population.

Key Concepts:

1. Null Hypothesis (H0): The null hypothesis is a statement that assumes there is no difference between the sample and the population. It is denoted as H0.

2. Alternative H

## Generate Quiz

In [212]:
def generate_quiz(topic, lesson):
    response = client.chat.completions.create(
        model="tiiuae/falcon-180B-chat",
        messages=[
            {"role": "system", "content": "You are a quiz bot. You create quizzes for students to test their knowledge on a specific lesson. You strictly follow the format provided and include the headers for the start of the two sections 'Multiple Choice Questions:' and 'True/False Questions:'."},
            {"role": "user", "content": 
f"""Create a quiz for the topic '{topic}' and lesson: {lesson[0:3000]}. 
The questions should be designed to test the student's understanding of the lesson. The quiz should be engaging and informative. Format the quiz as follows (Strickly follow the format):
Multiple Choice Questions:
1. [Question 1]
   A) [Option A]
   B) [Option B]
   C) [Option C]
   D) [Option D]

Answer: [Correct option letter]

2. [Question 2]
   A) [Option A]
   B) [Option B]
   C) [Option C]
   D) [Option D]

Answer: [Correct option letter]

3. [Question 3]
   A) [Option A]
   B) [Option B]
   C) [Option C]
   D) [Option D]

Answer: [Correct option letter]

4. [Question 4]
   A) [Option A]
   B) [Option B]
   C) [Option C]
   D) [Option D]

Answer: [Correct option letter]

5. [Question 5]
   A) [Option A]
   B) [Option B]
   C) [Option C]
   D) [Option D]

Answer: [Correct option letter]

True/False Questions:
6. [True/False statement]

Answer: [True/False correct answer]

7. [True/False statement]

Answer: [True/False correct answer]

8. [True/False statement]

Answer: [True/False correct answer]

9. [True/False statement]

Answer: [True/False correct answer]

10. [True/False statement]

Answer: [True/False correct answer]
"""}
        ],
    )
    return response

In [213]:
import time
def generate_quizzes(lessons):
    quizzes = {}
    for topic, lesson in lessons.items():
        response = generate_quiz(topic, lesson)
        quizzes[topic] = response.choices[0].message.content
        # time.sleep(5)
    return quizzes

In [214]:
QUIZ = generate_quizzes(BRUH)

In [215]:
for key, value in QUIZ.items():
    print(f"{value}")
    print("\n\n\n\n\n")

 Multiple Choice Questions:
1. What is calculus?
   A) A branch of mathematics that deals with the study of change and how things change over time
   B) A branch of mathematics that deals with the study of linear equations and their properties
   C) A branch of mathematics that deals with the study of data and its analysis
   D) A branch of mathematics that deals with the study of optimization algorithms

Answer: A

2. What is vectorization?
   A) The process of converting data into a vector format
   B) The process of converting data into a matrix format
   C) The process of converting data into a scalar format
   D) The process of converting data into a tensor format

Answer: A

3. What is linear algebra?
   A) A branch of mathematics that deals with the study of linear equations and their properties
   B) A branch of mathematics that deals with the study of change and how things change over time
   C) A branch of mathematics that deals with the study of data and its analysis
   D) A

In [216]:
print(QUIZ.keys())

dict_keys(['Calculus Overview', 'Hypothesis Testing', 'Vectorization', 'Linear Algebra', 'Calculus', 'Statistics'])


In [217]:
response = generate_quiz("Hypothesis Testing", BRUH["Hypothesis Testing"][0:3000])

In [218]:
print(len(BRUH["Hypothesis Testing"]))
print(response.usage)

4495
completion_tokens=637 prompt_tokens=1130 total_tokens=1767


In [219]:
questions_output = response.choices[0].message.content.replace("User:", "")

In [226]:
print([questions_output])

[' Multiple Choice Questions:\n\n1. What is hypothesis testing?\n   A) A statistical method used to make inferences about a population based on sample data\n   B) A method used to test the effectiveness of new drugs\n   C) A method used to test the safety of products\n   D) A method used to test the accuracy of financial models\n\nAnswer: A) A statistical method used to make inferences about a population based on sample data\n\n2. What is the null hypothesis?\n   A) A statement that assumes there is a difference between the sample and the population\n   B) A statement that assumes there is no difference between the sample and the population\n   C) A statement that assumes the sample is representative of the population\n   D) A statement that assumes the population is representative of the sample\n\nAnswer: B) A statement that assumes there is no difference between the sample and the population\n\n3. What is the alternative hypothesis?\n   A) A statement that assumes there is a differen

In [237]:
def convert_to_json(input_string):
    lines = input_string.strip().split('\n')
    result = {
        "Multiple Choice Questions": [],
        "True/False Questions": []
    }
    
    current_section = None
    question = None
    for line in lines:
        line = line.strip()
        if line.startswith('Multiple Choice Questions:'):
            current_section = "Multiple Choice Questions"
        elif line.startswith('True/False Questions:'):
            if question:
                result[current_section].append(question)
                question = None
            current_section = "True/False Questions"
        elif line and line[0].isdigit() and line[1] == '.':
            if question:
                result[current_section].append(question)
            question = {"Question": line}
        elif line.startswith('A)') or line.startswith('B)') or line.startswith('C)') or line.startswith('D)'):
            if 'Options' not in question:
                question['Options'] = []
            question['Options'].append(line)
        elif line.startswith('Answer:'):
            if current_section == "Multiple Choice Questions":
                question['Answer'] = line.split('Answer: ')[1]
            else:
                question['Answer'] = line.split('Answer: ')[1] == 'True'
        else:
            if question is not None:
                if 'Question' in question:
                    question['Question'] += ' ' + line
                else:
                    question['Question'] = line
    
    if question:
        result[current_section].append(question)
    
    return result

In [239]:
print(convert_to_json(questions_output))

{'Multiple Choice Questions': [{'Question': '1. What is hypothesis testing?  ', 'Options': ['A) A statistical method used to make inferences about a population based on sample data', 'B) A method used to test the effectiveness of new drugs', 'C) A method used to test the safety of products', 'D) A method used to test the accuracy of financial models'], 'Answer': 'A) A statistical method used to make inferences about a population based on sample data'}, {'Question': '2. What is the null hypothesis?  ', 'Options': ['A) A statement that assumes there is a difference between the sample and the population', 'B) A statement that assumes there is no difference between the sample and the population', 'C) A statement that assumes the sample is representative of the population', 'D) A statement that assumes the population is representative of the sample'], 'Answer': 'B) A statement that assumes there is no difference between the sample and the population'}, {'Question': '3. What is the alternati

In [241]:
Quizes = {}

In [242]:
for key, value in QUIZ.items():
    Quizes[key] = convert_to_json(value)

In [245]:
print(Quizes["Hypothesis Testing"])

{'Multiple Choice Questions': [{'Question': '1. What is hypothesis testing?  ', 'Options': ['A) A statistical method used to make inferences about a population based on sample data', 'B) A method used to test the effectiveness of new drugs', 'C) A method used to test the safety of products', 'D) A method used to test the accuracy of financial models'], 'Answer': 'A) A statistical method used to make inferences about a population based on sample data'}, {'Question': '2. What is the null hypothesis?  ', 'Options': ['A) A statement that assumes there is a difference between the sample and the population', 'B) A statement that assumes there is no difference between the sample and the population', 'C) A statement that assumes the sample is representative of the population', 'D) A statement that assumes the population is representative of the sample'], 'Answer': 'B) A statement that assumes there is no difference between the sample and the population'}, {'Question': '3. What is the alternati

In [246]:
Giga["quizzes"] = Quizes

In [248]:
print(Giga.keys())

dict_keys(['lesson_topics', 'lessons', 'quizzes'])
